# 対話制御の草案

- 初見判定
    - DBに発言者が登録されているか
        - 定型文をいくつか用意
- 登録済み
    - 配信時間内の2回目以降
        - 別の対話制御へ -> 
    - 1週間以内
        - また来てくれてありがとう
    - それ以外
        - お久しぶり



In [358]:
import sqlite3
import datetime



class UserInfoConnector:
    """
    接続するデータベース
    1. User
        uid : int
        name : str
        like : text
        last : datetime

    2. Comment
        cid : int
        uid : int
        content : text
        reply : text
        time : datetime
    
    """

    def __init__(self, db_path) -> None:
        self.connector = sqlite3.connect(db_path)
        self.cursor = self.connector.cursor()

        self.next_uid = -1
        self.next_cid = -1

        self.format_date = "%Y-%m-%d %H:%M:%S"

        # テーブルの作成(if not exist)
        self.cursor.execute('CREATE TABLE IF NOT EXISTS User(uid int, name varchar(512), like text, last datetime)')

        self.cursor.execute('CREATE TABLE IF NOT EXISTS Comment(cid int, uid int, content text, reply text, last datetime)')
    
    def datetime2str(self, dt:datetime)->str:
        return dt.strftime(self.format_date)
    
    def str2datetime(self, time:str)->datetime:
        return datetime.datetime.strptime(time, self.format_date)

    # テーブルリセット
    def reset_infomation(self):
        self.reset_User()
        self.reset_Comment()

    # ユーザリセット
    def reset_User(self):
        sql = 'DROP TABLE IF EXISTS User'
        self.cursor.execute(sql)
        sql = 'CREATE TABLE IF NOT EXISTS User(uid int, name varchar(512), like text, last datetime)'
        self.cursor.execute(sql)
        self.connector.commit()
    
    # コメントリセット
    def reset_Comment(self):
        sql = 'DROP TABLE IF EXISTS Comment'
        self.cursor.execute(sql)
        self.cursor.execute('CREATE TABLE IF NOT EXISTS Comment(cid int, uid int, content text, reply text, last datetime)')
        self.connector.commit()
    
    def serach_User_by_name(self, user_name:str):
        sql = 'SELECT * FROM User WHERE name="{0}"'.format(user_name)
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        return result
    
    def serach_User_by_uid(self, uid:int):
        sql = 'SELECT * FROM User WHERE uid="{0}"'.format(uid)
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        return result

    # ユーザの検索
    def is_exist_user(self, user_name:str):
        result = self.serach_User_by_name(user_name)

        if len(result)==1:
            return True
        elif len(result)==0:
            return False
        # 長さが2以上 -> 異常
        else:
            print("Number of matched users was more than 1")
            print(result)
            return True
        
    def get_user_id(self, user_name:str):
        result = self.serach_User_by_name(user_name)

        if len(result)==1:
            return int(result[0][0])
        elif len(result)==0:
            return -1
        # 長さが2以上 -> 異常
        else:
            print("Number of matched users was more than 1")
            return -1

    def register_user(self, user_name:str, comment:str, comment_time:datetime.datetime):
        time_str = self.datetime2str(comment_time)
        
        # 次の uid が -1 なら，探索
        if self.next_uid < 0:
            sql = 'SELECT uid FROM User ORDER BY uid' 
            self.cursor.execute(sql)
            result = self.cursor.fetchall()
            self.next_uid = int(result[-1][0]) + 1
            
        #  uid, name, like, last 
        sql = "INSERT INTO User VALUES({0}, '{1}', '{2}', '{3}')".format(
            self.next_uid, user_name, "", time_str,
        )
        self.cursor.execute(sql)
        self.connector.commit()

        # 次の番号へ更新
        self.next_uid += 1
    
    # コメント時刻の更新
    def update_comment_time(self, uid:int, comment_time:datetime.datetime):
        sql = "UPDATE User SET last='{0}' WHERE uid={1}".format(self.datetime2str(comment_time), uid)
        self.cursor.execute(sql)
        self.connector.commit()
    
    def register_comment(self, uid:int, comment:str, reply:str, comment_time:datetime.datetime):
        time_str = self.datetime2str(comment_time)
        if self.next_cid < 0:
            sql = 'SELECT cid FROM Comment ORDER BY cid' 
            self.cursor.execute(sql)
            result = self.cursor.fetchall()
            self.next_cid= int(result[-1][0]) + 1
        #  cid, uid, content, last 
        sql = "INSERT INTO Comment VALUES({0}, '{1}', '{2}', '{3}', '{4}')".format(
            self.next_cid, uid, comment, reply, time_str,
        )
        self.cursor.execute(sql)
        self.connector.commit()

        # 次の番号へ更新
        self.next_cid += 1
    
    def extract_context(self, uid:int, lst:datetime.datetime):
        time_str = self.datetime2str(lst)
        sql = "SELECT content, reply FROM Comment WHERE last>='{0}' AND uid={1}".format(time_str, uid)
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        return result

    def test_register_User(self):
        self.reset_User()
        self.cursor.execute(
            "INSERT INTO User VALUES(1, '福間創', 'シンセサイザ', '{0}')".format(self.datetime2str(datetime.datetime.now()))
        )
        self.cursor.execute(
            "INSERT INTO User VALUES(2, '平沢進', 'ギター', '{0}')".format(self.datetime2str(datetime.datetime.now()))
        )
        self.cursor.execute(
            "INSERT INTO User VALUES(3, '山田', 'ランニング', '{0}')".format(self.datetime2str(datetime.datetime.now()))
        )
        self.connector.commit()
    
    def test_register_Comment(self):
        """
        2. Comment
            cid : int
            uid : int
            content : text
            reply : text
            time : datetime
        """
        self.reset_Comment()

        self.cursor.execute(
            "INSERT INTO Comment VALUES(1, 1, '平沢さん怒ってます？', '何喧嘩？', '{0}')".format(self.datetime2str(datetime.datetime.now()))
        )
        self.cursor.execute(
            "INSERT INTO Comment VALUES(2, 2, 'それを聞くのは反則だよ', '絶対嘘じゃん！', '{0}')".format(self.datetime2str(datetime.datetime.now()))
        )
        self.connector.commit()

In [359]:
import datetime
import random
class Controller:
    def __init__(self):
        self.db_ctrller = UserInfoConnector("./example.db")

        self.live_start_time:datetime.datetime = datetime.datetime.now()
    
    # この関数を呼ぶ
    def reply(self, user_name:str, comment:str, comment_time:datetime.datetime):
        """
        user_name : str
            コメントしたユーザの名前
        comment : str
            コメント内容
        comment_time : datetime.datetime
            コメントした時刻
        """
        utt = ""
        # コメントした人(=ユーザ)が登録されているか
        if self.db_ctrller.is_exist_user(user_name):
            # 初見ではない
            # ユーザのid
            uid = self.db_ctrller.get_user_id(user_name)
            # この配信で初めてのコメントである
            tmp = self.db_ctrller.serach_User_by_uid(uid)
            prev_comment_time = self.db_ctrller.str2datetime(tmp[0][-1])
            # このユーザの最後のコメントが開始時刻よりも前ならば来場初コメント
            delta = self.live_start_time - prev_comment_time
            if int(delta.total_seconds()) >= 0 :
                utt += "{0}さん．こんにちは～．".format(user_name)
                # 前の発話との時差を確認
                tmp = self.db_ctrller.serach_User_by_uid(uid)
                prev_comment_time = self.db_ctrller.str2datetime(tmp[0][-1])
                delta = comment_time - prev_comment_time
                delta_day = int(delta.total_seconds())//(3600*24) 
                # print(int(delta.total_seconds())//(3600*24))
                # 一週間以内
                if delta_day <= 7:
                    # 一日も経ってないよ
                    if delta_day <= 0:
                        utt += "さっきぶり".format(delta_day)
                    else:
                        utt += "{0}日ぶり".format(delta_day)

                    candidate = [
                        "だね．やぁやぁ．",
                        "だね．また来てくれて嬉しいよ．",
                        "．元気？",
                        "にどうも～．",
                        "で合ってる？",
                        ]
                    utt += random.choice(candidate)
                else:
                    # お久しぶりセット
                    candidate = [
                        "お久しぶり！ ",
                        "また会えてうれしいよ！ ",
                        "元気だった？",
                        "生きてたんだぁ・",
                        "おひさ～．",
                        "俺のこと覚えてたんだ．"
                    ]
                    utt += random.choice(candidate)
            
            # 当該ライブでは2回目以降のコメント
            else:
                utt += "そうだっけ？　覚えてないや"
            
            # 最後の発話時刻を更新
            self.db_ctrller.update_comment_time(uid, comment_time)

        else:
            # 初見である
            utt += "{0}さん，はじめまして！".format(user_name)
            # ユーザを登録
            self.db_ctrller.register_user(user_name, comment, comment_time)
            # ユーザのid
            uid = self.db_ctrller.get_user_id(user_name)


        print(utt)
        # コメントを登録
        self.db_ctrller.register_comment(uid, comment, utt, comment_time)
    

In [360]:
ctrler = Controller()
ctrler.db_ctrller.test_register_User()
ctrler.db_ctrller.test_register_Comment()

In [361]:
time.sleep(0.5)
# ctrler.db_ctrller.register_user("yamada", "お前誰？", datetime.datetime.now())

In [362]:
t1 = '2022-03-20 22:18:48'
t2 = '2022-05-19 22:18:48'
tt1 = ctrler.db_ctrller.str2datetime(t1)
tt2 = ctrler.db_ctrller.str2datetime(t2)
import time

In [363]:
time.sleep(0.5)
ctrler.reply("yamada", "お前誰？", datetime.datetime.now())

yamadaさん，はじめまして！


In [364]:
time.sleep(0.5)
ctrler.reply("yamada", "ガチでお前誰？", datetime.datetime.now())

yamadaさん．こんにちは～．さっきぶりだね．また来てくれて嬉しいよ．


In [365]:
time.sleep(0.5)

In [366]:
ctrler.reply("tadaren", "初見です", datetime.datetime.now())

tadarenさん，はじめまして！


In [367]:
time.sleep(0.5)
ctrler.reply("tadaren", "はじめまして", datetime.datetime.now())

そうだっけ？　覚えてないや


In [368]:
time.sleep(1)
ctrler.reply("tadaren", "ふざけんな", datetime.datetime.now())

そうだっけ？　覚えてないや


In [369]:
time.sleep(1)
ctrler.reply("tadaren", "はじめまして", datetime.datetime.now())

そうだっけ？　覚えてないや


In [371]:
ctrler.db_ctrller.extract_context(4, ctrler.live_start_time)

[('お前誰？', 'yamadaさん，はじめまして！'),
 ('ガチでお前誰？', 'yamadaさん．こんにちは～．さっきぶりだね．また来てくれて嬉しいよ．')]